# Overview

This notebook is designed to be used alongside the [ProteinCartography pipeline](https://github.com/Arcadia-Science/ProteinCartography) to update the `plot_interactive.py` script to allow for custom overlays.

Once the `ProteinCartography` analysis has been completed and the resulting files are in the `ProteinCartography/output` folder, move back to the  `2023-actin-embedding` folder and to run this notebook to create custom plotting overlays.


## Setup

Import dependencies.


In [ ]:
import os
import pandas as pd
import sys
import plotly.express as px
import arcadia_pycolor as apc

sys.path.append("./../../ProteinCartography/ProteinCartography/")
from plot_interactive import (
    apply_coordinates,
    plot_interactive,
    generate_plotting_rules,
)


## Combine ProteinCartography results with results from the Actin Prediction pipeline

ProteinCartography produces an aggregated features file containing information for plotting as well as the relevant metadata. The Actin Prediction pipeline also produced a summary file. The first thing we need to do is combine these files.


In [ ]:
# Renames query column to protid in Actin Prediction results
all_outputs = pd.read_csv("../input/all_outputs_summarized.tsv", sep="\t")
all_outputs.insert(13, "protid", all_outputs["query"].values)

In [ ]:
# Merge outputs from each pipeline
aggregated_features = pd.read_csv(
    "../../ProteinCartography/Actin/output/clusteringresults/actin_aggregated_features_pca_umap.tsv",
    sep="\t",
)
combined_aggregated_features = aggregated_features.merge(
    all_outputs, how="outer", on="protid"
)

# Save file
combined_aggregated_features.to_csv("../output/combined_data.tsv", sep="\t", index=None)

display(combined_aggregated_features)


## Create plotting rules


In [ ]:
# Standard Arcadia plotting colors

arcadia_viridis = apc.Gradients["arcadia:viridis"].grad_nested_list
arcadia_viridis_r = apc.Gradients["arcadia:viridis_r"].grad_nested_list

arcadia_magma = apc.Gradients["arcadia:magma"].grad_nested_list
arcadia_magma_r = apc.Gradients["arcadia:magma_r"].grad_nested_list

arcadia_cividis = apc.Gradients["arcadia:cividis"].grad_nested_list
arcadia_cividis_r = apc.Gradients["arcadia:cividis_r"].grad_nested_list

arcadia_poppies = apc.Gradients["arcadia:poppies"].grad_nested_list
arcadia_poppies_r = apc.Gradients["arcadia:poppies_r"].grad_nested_list

arcadia_pansies = apc.Gradients["arcadia:pansies"].grad_nested_list
arcadia_pansies_r = apc.Gradients["arcadia:pansies_r"].grad_nested_list

arcadia_dahlias = apc.Gradients["arcadia:dahlias"].grad_nested_list
arcadia_dahlias_r = apc.Gradients["arcadia:dahlias_r"].grad_nested_list

arcadia_blues = apc.Gradients['arcadia:aegeans'].grad_nested_list
arcadia_reds = apc.Gradients['arcadia:reds_r'].grad_nested_list
arcadia_yellows = apc.Gradients['arcadia:yellows_r'].grad_nested_list


In [ ]:
# Set up annotation colors
from arcadia_pycolor import adjust_lightness

annotation_colors = {
    "LC00": apc.All["arcadia:aster"],  # actin
    "LC01": adjust_lightness(apc.All["arcadia:aster"], 0.9),  # actin
    "LC02": adjust_lightness(apc.All["arcadia:aster"], 1.1),  # actin
    "LC03": adjust_lightness(apc.All["arcadia:aster"], 1.2),  # actin
    "LC04": apc.All["arcadia:aegean"],  # ARP4
    "LC05": adjust_lightness(apc.All["arcadia:aster"], 1.3),  # actin
    "LC06": apc.All["arcadia:canary"],  # ARP7
    "LC07": apc.All["arcadia:mars"],  # ARP2
    "LC08": adjust_lightness(apc.All["arcadia:canary"], 1.2),  # ARP7
    "LC09": adjust_lightness(apc.All["arcadia:mars"], 1.1),  # ARP2
    "LC10": apc.All["arcadia:brightgrey"],  # ARP6
    "LC11": apc.All["arcadia:mint"],  # ARP3
    "LC12": adjust_lightness(apc.All["arcadia:mint"], 1.1),  # ARP3
    "LC13": adjust_lightness(apc.All["arcadia:aegean"], 1.15),  # ARP4
    "LC14": apc.All["arcadia:rose"],  # actin-like
    "LC15": adjust_lightness(apc.All["arcadia:brightgrey"], 0.9),  # ARP6
    "LC16": adjust_lightness(apc.All["arcadia:aster"], 1.4),  # actin
    "LC17": adjust_lightness(apc.All["arcadia:mint"], 0.9),  # ARP3
    "LC18": adjust_lightness(apc.All["arcadia:aster"], 1.5),  # actin
    "LC19": apc.All["arcadia:lime"],  # ARP1
    "LC20": adjust_lightness(apc.All["arcadia:mars"], 0.9),  # ARP2
    "LC21": adjust_lightness(apc.All["arcadia:mint"], 0.85),  # ARP3
    "LC22": adjust_lightness(apc.All["arcadia:seaweed"], 0.9),  # ARP6
    "": apc.All["arcadia:seaweed"],
}


In [ ]:
# Import plotting rules
plotting_rules = generate_plotting_rules(
    taxon_focus="euk", keyids=["P60709"], version="current"
)


In [ ]:
# Define plotting rules
custom_plotting_rules = {
    "lon_fraction_matching": {
        "type": "continuous",
        "color_scale": arcadia_blues,
        "cmin": 0,
        "cmax": 1,
        "textlabel": "Longitudinal contact conservation",
    },
    "lat_fraction_matching": {
        "type": "continuous",
        "color_scale": arcadia_reds,
        "cmin": 0,
        "cmax": 1,
        "textlabel": "Lateral contact conservation",
    },
    "atp_fraction_matching": {
        "type": "continuous",
        "color_scale": arcadia_yellows,
        "cmin": 0,
        "cmax": 1,
        "textlabel": "ATP contact conservation",
    },
    "w_avg_contacts": {
        "type": "continuous",
        "color_scale": arcadia_viridis,
        "cmin": 0,
        "cmax": 1,
        "textlabel": "Total polymerization conservation",
    },
    "LeidenCluster": {
        "type": "categorical",
        "color_dict": annotation_colors,
        "apply": lambda x: str(x),
        "textlabel": "Top Annotation",
        "fillna": "",
    },
}


In [ ]:
# Merge plotting rules
combined_plotting_rules = plotting_rules | custom_plotting_rules
combined_plotting_rules

## Plotting

We created the interactive plot using the `plot_interactive` command from the `plot_interactive.py` script within ProteinCartography.


In [ ]:
plot_interactive(
    coordinates_file="../output/combined_data.tsv",
    plotting_rules=combined_plotting_rules,
    output_file="../output/aggregated_maps.html",
    keyids=["P60709"],
    plot_width = 700,
    plot_height = 750,
    show=False,
)